In [22]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras import backend as K, models
import os

Data Preprocessing

In [23]:
# === CONFIG ===
sequence_length = 12 # 12 hours of data
input_csv = r'C:\Users\brian\Documents\Project\data\processed\ieee13_multitype_fdia.csv'

df = pd.read_csv(input_csv)
df = df.drop(columns=["timestamp", "fdia_target_bus", "day"])
df = df.sort_values(by=["hour"]).reset_index(drop=True)

label_fdia = df["fdia"].values
label_fdia_type = df["fdia_type"].values
features = df.drop(columns=["fdia", "fdia_type"]).values

scaler = MinMaxScaler()
features_scaled = scaler.fit_transform(features)

In [24]:
# === BUILD SEQUENCES ===
X, y_fdia, y_fdia_type = [], [], []
for i in range(len(features_scaled) - sequence_length):
    X_seq = features_scaled[i:i + sequence_length]
    y_label_fdia = label_fdia[i + sequence_length - 1]  # Binary FDIA
    y_label_type = label_fdia_type[i + sequence_length - 1]  # Multi-class
    X.append(X_seq)
    y_fdia.append(y_label_fdia)
    y_fdia_type.append(y_label_type)

X = np.array(X)
y_fdia = np.array(y_fdia)
y_fdia_type = np.array(y_fdia_type)

X_train, X_test, y_train, y_test = train_test_split(X, y_fdia, test_size=0.2, random_state=42)
X_train_type, X_test_type, y_train_type, y_test_type = train_test_split(X, y_fdia_type, test_size=0.2, random_state=42)


In [25]:
# === TF.DATA PIPELINE ===
def create_tf_dataset(X, y, batch_size=32, shuffle=True):
    ds = tf.data.Dataset.from_tensor_slices((X, y))
    if shuffle:
        ds = ds.shuffle(buffer_size=len(X))
    return ds.batch(batch_size).prefetch(tf.data.AUTOTUNE)

train_ds = create_tf_dataset(X_train, y_train)
test_ds = create_tf_dataset(X_test, y_test, shuffle=False)

# === PREVIEW ===
print("Data prepared and saved.")
print("X shape:", X.shape)
print("Train set:", X_train.shape, y_train.shape)
print("Test set:", X_test.shape, y_test.shape)
print("TF dataset example:", next(iter(train_ds))[0].shape)

Data prepared and saved.
X shape: (156, 12, 18)
Train set: (124, 12, 18) (124,)
Test set: (32, 12, 18) (32,)
TF dataset example: (32, 12, 18)


Build the Model

In [26]:
import tensorflow as tf
from tensorflow.keras import layers, models

# === LSTM Encoder ===
def build_encoder(input_dim, timesteps, intermediate_dim, latent_dim):
    inputs = tf.keras.Input(shape=(timesteps, input_dim))

    # Set initializer for LSTM weights to avoid gradient issues.
    kernel_initializer = tf.keras.initializers.GlorotUniform(seed=42)
    recurrent_initializer = tf.keras.initializers.Orthogonal(seed=42)

    x = layers.LSTM(intermediate_dim, kernel_initializer=kernel_initializer,
                    recurrent_initializer=recurrent_initializer)(inputs)
    
    # Mean and Variance in Latent Space
    z_mean = layers.Dense(latent_dim, kernel_initializer=kernel_initializer)(x)
    z_log_sigma = layers.Dense(latent_dim, kernel_initializer=kernel_initializer)(x)
    
    # Sampling Function
    def sampling(args):
        z_mean, z_log_sigma = args
        epsilon = tf.random.normal(shape=(tf.shape(z_mean)[0], latent_dim), mean=0., stddev=1.0, seed=42)
        return z_mean + tf.exp(z_log_sigma / 2) * epsilon

    # Wrap Sampling Function in a Lambda Layer
    z = layers.Lambda(sampling, output_shape=(latent_dim,))([z_mean, z_log_sigma])

    # Build the Encoder Model
    encoder = models.Model(inputs, [z_mean, z_log_sigma, z], name="encoder")
    return encoder

# === LSTM Decoder ===
def build_generator(latent_dim, timesteps, output_dim, intermediate_dim):
    inputs = tf.keras.Input(shape=(latent_dim,))
    # Set initializer for LSTM weights to avoid gradient issues.
    kernel_initializer = tf.keras.initializers.GlorotUniform(seed=42)
    recurrent_initializer = tf.keras.initializers.Orthogonal(seed=42)

    # Adjust the latent input to match LSTM input shape by repeating the latent vector 'timesteps' times.
    repeated_z = layers.RepeatVector(timesteps)(inputs)

    # Single LSTM layer to generate the sequence, with 'return_sequences=True' to get output at each time step.
    x = layers.LSTM(intermediate_dim, return_sequences=True, kernel_initializer=kernel_initializer, recurrent_initializer=recurrent_initializer)(repeated_z)

    # Output layer: Generates time series data.
    # TimeDistributed is used to ensure that Dense is applied at every time step.
    # Sigmoid activation is used to constrain the output between 0 and 1.
    outputs = layers.TimeDistributed(layers.Dense(output_dim, activation='sigmoid', kernel_initializer=kernel_initializer))(x)

    # Build the generator model.
    generator = models.Model(inputs, outputs, name='generator')
    return generator

# === Discriminator ===
def build_discriminator(timesteps, input_dim, intermediate_dim):
    inputs = tf.keras.Input(shape=(timesteps, input_dim))
    # Set initializer for LSTM weights to avoid gradient issues.
    kernel_initializer = tf.keras.initializers.GlorotUniform(seed=42)
    recurrent_initializer = tf.keras.initializers.Orthogonal(seed=42)

    # LSTM layer: Processes the input sequence and returns outputs at each time step.
    lstm_out = layers.LSTM(intermediate_dim, return_sequences=True, kernel_initializer=kernel_initializer, recurrent_initializer=recurrent_initializer)(inputs)

    # Only the output from the last time step is used for final classification.
    # This Dense layer applies a sigmoid activation to return a probability (0 to 1).
    final_output = layers.Dense(1, activation='sigmoid', kernel_initializer=kernel_initializer)(lstm_out[:, -1, :])

    # Build the discriminator model, which outputs both the final_output and the lstm_out (Calculating the loss function is required).
    discriminator = models.Model(inputs, outputs=[final_output, lstm_out], name="discriminator")
    return discriminator

Train

In [27]:
# === PARAMETERS ===
# These will be updated based on actual data dimensions in the next cell
latent_dim = 10 # Dimensionality of the latent space.
# timesteps and input_dim will be determined from data shape
intermediate_dim = 60 # Hidden units in the LSTM Layer.

In [28]:
# Check current shapes
print("X_train shape:", X_train.shape)
print("X_test shape:", X_test.shape)

# The data already has the correct 3D shape (samples, timesteps, features)
# No need to reshape if it's already 3D, just update the parameters
if len(X_train.shape) == 3:
    # Update parameters to match actual data dimensions
    timesteps = X_train.shape[1]  # sequence length (12)
    input_dim = X_train.shape[2]  # number of features per timestep
    print(f"Updated parameters: timesteps={timesteps}, input_dim={input_dim}")
else:
    # If it's 2D, we need to determine how to reshape
    total_features = X_train.shape[1]
    sequence_length = 12  # from config
    input_dim = total_features // sequence_length
    
    X_train = X_train.reshape(X_train.shape[0], sequence_length, input_dim)
    X_test = X_test.reshape(X_test.shape[0], sequence_length, input_dim)
    
    timesteps = sequence_length
    print(f"Reshaped data: timesteps={timesteps}, input_dim={input_dim}")
    print("New X_train shape:", X_train.shape)
    print("New X_test shape:", X_test.shape)

X_train shape: (124, 12, 18)
X_test shape: (32, 12, 18)
Updated parameters: timesteps=12, input_dim=18


In [29]:
from tensorflow.keras import backend as K

def kl_divergence_loss(z_mean, z_log_sigma):
    kl_loss = -0.5 * K.sum(1 + z_log_sigma - K.square(z_mean) - K.exp(z_log_sigma), axis=-1)
    return kl_loss

def reconstruction_loss(discriminator_hidden_output, discriminator_hidden_output_1):
    return tf.reduce_mean(tf.square(discriminator_hidden_output - discriminator_hidden_output_1), axis=-1)

def encoder_loss(z_mean, z_log_sigma, discriminator_hidden_output, discriminator_hidden_output_1):
    kl_loss = kl_divergence_loss(z_mean, z_log_sigma)
    re_loss = reconstruction_loss(discriminator_hidden_output, discriminator_hidden_output_1)
    encoder_loss = tf.reduce_mean(kl_loss) + tf.reduce_mean(re_loss)
    return encoder_loss

def discriminator_loss(discriminator_output_1, discriminator_output_2, discriminator_output_real):
    discriminator_loss = -K.mean(K.log(1 - discriminator_output_1) + K.log(1 - discriminator_output_2) + K.log(discriminator_output_real))
    return discriminator_loss

def generator_loss(discriminator_output_1, discriminator_output_2, discriminator_hidden_output, discriminator_hidden_output_1):
    adv_loss = -K.mean(K.log(discriminator_output_1 + 1e-10) + K.log(discriminator_output_2 + 1e-10))
    re_loss = reconstruction_loss(discriminator_hidden_output, discriminator_hidden_output_1)
    generator_loss = adv_loss + tf.reduce_mean(re_loss)
    return generator_loss

In [30]:
encoder_optimizer = tf.keras.optimizers.Adam(0.001)
generator_optimizer = tf.keras.optimizers.Adam(0.001)
discriminator_optimizer = tf.keras.optimizers.Adam(0.001)

encoder = build_encoder(input_dim, timesteps, intermediate_dim, latent_dim)
generator = build_generator(latent_dim, timesteps, input_dim, intermediate_dim)
discriminator = build_discriminator(timesteps, input_dim, intermediate_dim)

@tf.function
def train_step(x):
    # Persistent GradientTape to allow multiple gradient calculations
    with tf.GradientTape(persistent=True) as tape:
        # Forward pass through the encoder
        # Encode the input x and obtain z_mean, z_log_sigma, and the sampled latent vector z
        z_mean, z_log_sigma, z = encoder(x, training=True)

        # Generate reconstructed data using the sampled latent vector z
        reconstructed_x1 = generator(z, training=True)

        # Generate a random latent vector z_random and use it to create another generated data
        z_random = tf.random.normal(shape=(1, latent_dim), seed=42)
        reconstructed_x2 = generator(z_random, training=True)

        # Discriminator output for real input data
        discriminator_output_real, discriminator_hidden_output = discriminator(x, training=True)

        # Discriminator output for data reconstructed from z
        discriminator_output_1, discriminator_hidden_output_1 = discriminator(reconstructed_x1, training=True)

        # Discriminator output for data generated from random latent vector z_random
        discriminator_output_2, discriminator_hidden_output_2 = discriminator(reconstructed_x2, training=True)

        # Compute the losses
        # Encoder loss combines KL divergence and reconstruction loss
        en_loss = encoder_loss(z_mean, z_log_sigma, discriminator_hidden_output, discriminator_hidden_output_1)

        # Generator loss combines adversarial loss and reconstruction loss
        gen_loss = generator_loss(discriminator_output_1, discriminator_output_2, discriminator_hidden_output, discriminator_hidden_output_1)

        # Discriminator loss measures how well it distinguishes between real and generated data
        dis_loss = discriminator_loss(discriminator_output_1, discriminator_output_2, discriminator_output_real)

        # Backpropagation step
        # Calculate gradients for each model's parameters

        # Calculate gradients for encoder based on encoder loss
        gradients_of_encoder = tape.gradient(en_loss, encoder.trainable_variables)

        # Calculate gradients for generator based on generator loss
        gradients_of_generator = tape.gradient(gen_loss, generator.trainable_variables)

        # Calculate gradients for discriminator based on discriminator loss
        gradients_of_discriminator = tape.gradient(dis_loss, discriminator.trainable_variables)

        # Apply gradients to update the model parameters
        # Update the encoder weights
        encoder_optimizer.apply_gradients(zip(gradients_of_encoder, encoder.trainable_variables))

        # Update the generator weights
        generator_optimizer.apply_gradients(zip(gradients_of_generator, generator.trainable_variables))

        # Update the discriminator weights
        discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, discriminator.trainable_variables))

        # Return the losses for monitoring
        return en_loss, gen_loss, dis_loss

# Training parameters
batchsize = 256
epochs = 25

# Training loop
for epoch in range(epochs):
    for i in range(0, len(X_train), batchsize):
        # Get a batch of training data
        x = X_train[i:i+batchsize]
        # Perform a training step and compute the losses
        loss_values = train_step(x)

In [32]:
# The generator takes the latent variables from the encoder output as input
encoded_inputs = encoder.input # Get the input of the encoder (i.e., the original input data)
encoded_outputs = encoder.output[2] # Get the third output of the encoder (latent vector z)

# Pass the latent vector z from the encoder to the generator, producing the generator's output
generated_outputs = generator(encoded_outputs) # Use the generator to create the reconstructed data

# The input of VAE_GENERATOR_MODEL is the encoder's input, and the output is the generator's output.
VAE_GENERATOR_MODEL = models.Model(encoded_inputs, generated_outputs)

# Extract the input and the first output from the discriminator
discriminator_inputs = discriminator.input

# Get the first output of the discriminator, which usually represents the probability that the input is real
discriminator_final_output = discriminator.output[0] # Access the first output layer of the discriminator

# Construct a discriminator model that only includes the final output.
DISCRIMINATOR = models.Model(inputs=discriminator_inputs, outputs=discriminator_final_output)

def compute_anomaly_score(X, alpha):
    # Use the VAE-GENERATOR model to generate data based on the input X
    generated_output = VAE_GENERATOR_MODEL.predict(X)

    # Calculate the reconstruction error (how different the input X is from the generated output)
    # Reconstruction error is calculated as the mean absolute error between the original input and the generated output
    reconstruction_error = np.mean(np.abs(X - generated_output), axis=(1, 2))

    # Pass the input data through the simplified discriminator model to get the first output (usually real/fake probability)
    discriminator_final_output = DISCRIMINATOR(X) # Get the first output from the discriminator

    # Ensure that the discriminator output is a numpy array
    if isinstance(discriminator_final_output, tf.Tensor):
        discriminator_final_output = discriminator_final_output.numpy().flatten() # Convert to a numpy array and flatten it if needed

    # Compute the anomaly score using a weighted combination of reconstruction error and discriminator output
    anomaly_score = np.abs((1 - alpha) * reconstruction_error - alpha * discriminator_final_output)

    # Return the calculated anomaly score
    return anomaly_score

def average_scores(anomaly_scores, window_size, step_size, original_length):
    num_points = original_length # The number of points in the original time series
    avg_scores = np.zeros((num_points,)) # Array to accumulate anomaly scores for each point
    counts = np.zeros((num_points,)) # Array to count how many times each point is included in a sub-sequence

    # Loop through each sub-sequence's anomaly score
    for i, score in enumerate(anomaly_scores):
        # Determine the starting and ending indices of the current sub-sequence within the original time series
        start_idx = i * step_size
        end_idx = start_idx + window_size

        # Accumulate the anomaly score for each point in the current sub-sequence
        avg_scores[start_idx:end_idx] += score

        # Increment the count for each point to track how many times it is covered by sub-sequences
        counts[start_idx:end_idx] += 1

    # Avoid division by zero in case some points are not covered by any sub-sequence
    counts[counts == 0] = 1

    # Compute the average anomaly score by dividing the accumulated score by the count for each point
    avg_scores /= counts

    return avg_scores

alpha = 0.3
window_size = 10
step_size = 3

anomaly_scores = compute_anomaly_score(X_test, alpha)
avg_scores = average_scores(anomaly_scores, window_size, step_size, len(X_test))
avg_scores

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 534ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 534ms/step


array([0.11910819, 0.11910819, 0.11910819, 0.11514522, 0.11514522,
       0.11514522, 0.12122863, 0.12122863, 0.12122863, 0.12354461,
       0.12502342, 0.12502342, 0.11167277, 0.11183628, 0.11183628,
       0.0997867 , 0.08858377, 0.08858377, 0.09835184, 0.08763827,
       0.08763827, 0.08472872, 0.08909802, 0.08909802, 0.09901838,
       0.11081186, 0.11081186, 0.09425934, 0.08312711, 0.08312711,
       0.07545411, 0.07527212])

In [35]:
# X_test_partial = X_test.loc[2700:3300]

# true_labels = X_test['label'].values
# true_labels_partial = true_labels.loc[2700:3300]

# sub_test = create_subsequences(X_test_partial, window_size, step_size)
# sub_test = sub_test.reshape(sub_test.shape[0], sub_test.shape[1], 1)

# num_anomalies = (true_labels_partial == 1).sum()
# num_anomalies

IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices

In [33]:
from sklearn import metrics

# Function to select the best threshold:
# Takes candidate thresholds, average anomaly scores, and true labels as input, returns the best threshold and corresponding F1 score.
def select_threshold(candidate_thresholds, avg_scores1, true_labels_partial):
    best_f1_score = -1 # Initialize the best F1 score as -1, for comparison purposes.
    best_threshold = None # Initialize the best threshold as None.

    # Iterate through each candidate threshold.
    for threshold in candidate_thresholds:
        # Convert average anomaly scores to predicted labels by comparing them to the threshold
        #(greater than threshold = 1, indicating anomaly; less than or equal = 0, indicating normal).
        predicted_labels = (avg_scores1 > threshold).astype(int)

        # Compute precision, recall, and F1 score for the current threshold.
        precision, recall, f1_score, _ = metrics.precision_recall_fscore_support(
            true_labels_partial, predicted_labels, average='binary'
        )

        # If the F1 score for the current threshold is higher than the best F1 score so far, update the best F1 score and threshold.
        if f1_score > best_f1_score:
            best_f1_score = f1_score
            best_threshold = threshold

    return best_threshold, best_f1_score # Return the best threshold and the corresponding F1 score.

# original_length1 is the length of X_test_partial.
original_length1 = len(X_test)

# Compute anomaly scores.
anomaly_scores1 = compute_anomaly_score(X_test, alpha)

# Average the scores from sliding windows to get the average anomaly score for each time point.
avg_scores1 = average_scores(anomaly_scores1, window_size, step_size, original_length1)

# Calculate the minimum and maximum average anomaly scores to define the range for candidate thresholds.
min_avg_scores1 = np.min(avg_scores1)
max_avg_scores1 = np.max(avg_scores1)

# Generate 10 candidate thresholds evenly spaced between the minimum and maximum average anomaly scores.
num_candidates = 10
candidate_thresholds = np.linspace(min_avg_scores1, max_avg_scores1, num_candidates)

# Call the select_threshold function to select the best threshold and the corresponding best F1 score.
best_threshold, best_f1_score = select_threshold(candidate_thresholds, avg_scores1, true_labels_partial)

test_predicted_labels = (avg_scores1 > best_threshold).astype(int)
test_precision, test_recall, test_f1_score, _ = metrics.precision_recall_fscore_support(
    true_labels, test_predicted_labels, average='binary'
)
print("Best Threshold:", best_threshold)
print("Best F1 Score:", best_f1_score)
print("Test Precision:", test_precision)
print("Test Recall:", test_recall)
print("Test F1 Score:", test_f1_score)
#

NameError: name 'X_test_partial' is not defined